In [1]:
!pip install ebaysdk

In [2]:
MyToken = "AgAAAA**AQAAAA**aAAAAA**3+rCXA**nY+sHZ2PrBmdj6wVnY+sEZ2PrA2dj6ANloKlAZGDow2dj6x9nY+seQ**6voFAA**AAMAAA**y4HII3vHsCqPOFD5XxwoxNBZPrFz0chf4UdjnQQEYHhCuvyjrC5mGbttTLgx6LCwx9DwGig0yMlzavIuGCbYRzdqhAIA/PvSpqDjzdK8jRh1r8SIYFzvHP1sR5S9betmx8N4nJ7UvVUj5cDAHIqSkiQycQ4G66ADMQd9gWyRv39h0mVK+K+c0Z/Kkz7zMtROiiD4Y0dYfDuBCaJY6cTaTZtmnCBRy0kKJHnz9yCgnrWS6DCCmSuzNdLZq0NtOrA0NRslUQqHItrhqjTbBQfhq2NdJu1AteQmc25IkAkkSXjzRjOAkF7pGZhUOSMA2A6JjSOffSPIEKfYSorpC8CAiu8zeE3t4HqRPN4kbPbRKZ6//JXkMDhe7Aj5T+lU0UdE9l5ojtuH5nfm9af0ewuQHNFBTCTAT7gLneFJZSc6eAupZjfoktaVcNmsVVPGERIly9oOfspbQvMc+OHjWfHOyg2WXBzSioq2MhLhtDbOn7SUQdW2pVo27+xpwN7QZ4824oUr7tj8uy+P3VGFvLkfl2gQulg+H8NYty8uAxQN85YvSc5Vz3Id/vRIFPy4Sss3xNH619b0/XFCEZ9iJrxJYVLZw4Ictbbe2qYrbctrRehKlMtThEWZiF1N/uBB6iympClsuKh1OenarCieWOhjtL1oFG5NFD7cGoPLsMQeE337BhN6o6t90OBhqZLVMhCgovrDF0gSVW/bkYyGzquBUorJ3Oi+6vyXnGLTZorPqHC63aXwZRW81hqarLN0qhsa"
MyCertID='PRD-eaa024340831-68cd-4d05-80cc-37ae'
MyDevID='da60db7a-fb35-4b1d-af99-e1d74f695fd7'
MyAppID='SMojtaba-Analyzin-PRD-2eaa02434-919db739'
Entry_Per_Page  = 100
fridges_details = r"fridges_info"
data_dir = "data"
Daily_Call = 60

In [3]:
import os
from datetime import datetime
import codecs
from ebaysdk.trading import Connection as Trading
import ebaysdk
from ebaysdk.finding import Connection as finding
from ebaysdk.shopping import Connection as Shopping
from bs4 import BeautifulSoup
import requests


In [25]:
def write_fridges_info(item, header=False):
        global fridges_details
        global data_dir
        if not header:
            f = codecs.open(data_dir + os.path.sep +fridges_details + datetime.now()
                        .strftime("-%Y-%m") + '.csv', "a", encoding="utf8")
            row_csv = "{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13}\r\n".format(
                item["itemId"],
                item["title"].replace(",", "^") if item["title"] is not None else item["title"],
                item['categoryName'].replace(",", "^") if item["categoryName"] is not None else item["categoryName"],
                item["location"].replace(",", "^") if item["location"] is not None else item["location"],
                item["country"],
                item["model"].replace(",", "^") if item["model"] is not None else item["model"],
                item["brand"].replace(",", "^") if item["brand"] is not None else item["brand"],
                item["capacity"],
                item["condition"],
                item["deliveryTime"].replace(",", "^") if item["deliveryTime"] is not None else item["deliveryTime"],
                item["dispatchTimeMax"],
                item["price"],
                item["currency"],
                item["viewItemURL"],
            )
        else:
            f = codecs.open(data_dir + os.path.sep +fridges_details + datetime.now()
                        .strftime("-%Y-%m") + '.csv', "w", encoding="utf8")
            row_csv = "{0},{1},{2},{3},{4},{5},{6},{7},{8},{9},{10},{11},{12},{13}\r\n".format("ID", "Title", "Category", "Location", "Country",
                                                                        "Model","Brand","Capacity","Condition","DeliveryTime","DispatchTimeMax","Price",
                                                                                          "Currency","URL")
        f.write(row_csv)
        f.close()
        
write_fridges_info({},header=True)

In [5]:
def issueMainQuery(pgn=1):
    global Daily_Call
    api = finding(siteid='EBAY-AU', appid='SMojtaba-Analyzin-PRD-2eaa02434-919db739',config_file=None)
    api.execute('findItemsAdvanced', {
        'keywords': '',
        'categoryId': ['20713', '71262'],
        'itemFilter': [
            {'name': 'Condition', 'value': ['Used','New']},

        ],

        'paginationInput': {
            'entriesPerPage': Entry_Per_Page,
            'pageNumber': pgn
        },
        'sortOrder': 'CurrentPriceHighest'
    })
    Daily_Call += 1
    return api.response.dict()

In [6]:
def getExtraInfoAPI(item):
    global Daily_Call
    Daily_Call +=1
    api = Trading(certid=MyCertID,devid=MyDevID,token=MyToken, appid=MyAppID, config_file=None)
    response = api.execute('GetItem', {'ItemID':item["itemId"], 'IncludeItemSpecifics' : True })
    dictstr = response.dict()
    
    Brand=None
    Capacity = None
    Model = None
    DeliveryTime=None
    DispatchTimeMax=None
    
    
    if "ShippingServiceOptions" in dictstr["Item"]["ShippingDetails"].keys():
        DeliveryTime = dictstr["Item"]["ShippingDetails"]["ShippingServiceOptions"]["ShippingTimeMax"]
    if "DispatchTimeMax" in dictstr["Item"].keys():
        DispatchTimeMax = dictstr["Item"]["DispatchTimeMax"]
    for v in dictstr["Item"]["ItemSpecifics"]["NameValueList"] :
        if v["Name"] == "Brand" :
            Brand = v["Value"]
        elif v["Name"] == "Model" :
            Model = v["Value"]
        elif v["Name"] == "Capacity" :
            Capacity = v["Value"]
    item["brand"] = Brand
    item["capacity"]  =Capacity
    item["model"] = Model
    item["deliveryTime"] = DeliveryTime
    item["dispatchTimeMax"] = DispatchTimeMax
    
    #display(dictstr)

In [7]:
def getExtraInfoBS4(item):
    page = requests.get(item["viewItemURL"])
    soup = BeautifulSoup(page.content, 'html.parser')
    item_specific = soup.select('div.itemAttr td')
    
    Brand=None
    Condition=None
    Capacity = None
    Model = None
    DeliveryTime=None
    DispatchTimeMax=None
    
    for idx,tag in enumerate(item_specific[::2]):  
        #print(idx,tag.text.strip())

#         if tag.text.strip().startswith("Condition"):
#             txt = item_specific[idx*2+1].text.strip()
#             if  txt.startswith("Brand New"):
#                 Condition ="Brand New"
#             elif txt.startswith("Used"):
#                 Condition ="Used"    
        if tag.text.strip().startswith("Brand") :
            Brand = item_specific[idx*2+1].text.strip()
        elif tag.text.strip().startswith("Model") :
            Model = item_specific[idx*2+1].text.strip()
        elif tag.text.strip().startswith("Capacity") :
            Capacity = item_specific[idx*2+1].text.strip()


    dl_time =  soup.select('span.vi-acc-del-range')
    if len(dl_time)>0:
        DeliveryTime = dl_time[0].text

    dispatch_time =  soup.select('span.sh-DlvryDtl')
    if len(dispatch_time)>0:
        DispatchTimeMax = dispatch_time[0].text[20:22]

    item["deliveryTime"] = DeliveryTime
    item["dispatchTimeMax"] = DispatchTimeMax
    item["brand"] = Brand
    item["capacity"]  =Capacity
    item["model"] = Model
    
    return

In [24]:
def getExtraInfo(item):
    global Daily_Call
    if Daily_Call >= 4999:
        getExtraInfoBS4(item)
    else: 
        try : 
            getExtraInfoAPI(item)
        except:
            getExtraInfoBS4(item)

In [9]:
def extractCapacity(item):
    tokens = item["title"].split()
    cap = 0 
    for t in tokens :
        if t.endswith("Lt") or t.endswith("lt") :
            try : 
                cap +=int(t[:-2])
            except:
                pass
        elif t.endswith("L"):   
            try : 
                cap +=int(t[:-1])
            except:
                pass
    if cap>0 :
        item["capacity"] = cap
    return    
        

In [ ]:
dictstr = issueMainQuery()
totalCount = int(dictstr["paginationOutput"]["totalEntries"])
print("Total Entries : {}".format(totalCount))
print("*"*30)
page_count = totalCount//100+1
item_list =[]
for pgn in range(1,page_count+1) :
    print("-"*30)
    print("Current Page : {}".format(pgn))
    itm_cnt = 1
    print("-"*30)
    dictstr = issueMainQuery(pgn)
    try : 
        for item in dictstr['searchResult']['item']:
            print("Current Item Index  : {}".format( (pgn-1)*Entry_Per_Page + itm_cnt))
            item_data={}
            print('ItemID: {}'.format(item['itemId']))
            item_data["itemId"] = item['itemId']
            print('Title: {}'.format(item['title']))
            item_data["title"] = item['title']
            print('CategoryID: {}'.format(item['primaryCategory']['categoryId']))
            print('CategoryName: {}'.format(item['primaryCategory']['categoryName']))
            item_data["categoryName"] = item['primaryCategory']['categoryName']
            print('Location: {}'.format(item['location']))
            item_data["location"] = item['location']
            print('Country: {}'.format(item['country']))
            item_data["country"] = item['country']
            item_data["viewItemURL"] = item["viewItemURL"]
            print('Condition: {}'.format(item['condition']))
            item_data["condition"] = item["condition"]['conditionDisplayName']
            item_data["price"] = item["sellingStatus"]["convertedCurrentPrice"]["value"]
            item_data["currency"] =item["sellingStatus"]["convertedCurrentPrice"]["_currencyId"]
            itm_cnt +=1
            #write_fridges_info(item_data)
            item_list.append(item_data)
            print("$"*30)
    except:
        if "errorMessage" in dictstr.keys() :
            if dictstr["errorMessage"]["error"]["errorId"] == '61' :
                print("@@"*20 + "Page Limit Reached!" + "@@"*20)
        break        
for item in item_list : 
    getExtraInfo(item)
    if item["capacity"]==None :
            extractCapacity(item)
    write_fridges_info(item)        
    display(item)

Total Entries : 13712
******************************
------------------------------
Current Page : 1
------------------------------
Current Item Index  : 1
ItemID: 123344920311
Title: ASKO Pro Series Refrigerator and wine fridge NEW IN BOXES
CategoryID: 20713
CategoryName: Refrigerators
Location: Essendon,VIC,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 2
ItemID: 382766518976
Title: Commercial Dairy / Fruit & Veg Fridge in Great Condition
CategoryID: 20713
CategoryName: Refrigerators
Location: North Albury,NSW,Australia
Country: AU
Condition: {'conditionId': '3000', 'conditionDisplayName': 'Used'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 3
ItemID: 273670120203
Title: NEW Liebherr ECBN5066LH 462L Integrated Bottom Mount Fridge
CategoryID: 20713
CategoryName: Refrigerators
Location: Sydney,NSW,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'B

Current Item Index  : 101
ItemID: 233141255698
Title: Mitsubishi 500L French/Multi Drawer Fridge - MR-WX500C-W-A
CategoryID: 20713
CategoryName: Refrigerators
Location: West Footscray,VIC,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 102
ItemID: 264280787378
Title: Skipio SGM-25 Upright Fridge - 510L
CategoryID: 20713
CategoryName: Refrigerators
Location: Parramatta,NSW,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 103
ItemID: 264280787378
Title: Skipio SGM-25 Upright Fridge - 510L
CategoryID: 20713
CategoryName: Refrigerators
Location: Parramatta,NSW,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 104
ItemID: 264280787378
Title: Skipio SGM-25 Upright Fridge - 510L
CategoryID: 20713
Categor

Current Item Index  : 201
ItemID: 401741996730
Title: Samsung RS5FK6608SL/EG RS5FK6608SL side-by-side refrigerator Freestanding
CategoryID: 71262
CategoryName: Mini Fridges
Location: Italy
Country: IT
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 202
ItemID: 282461741379
Title: Vitrifrigo Stainless steel Drawer Refrigerator + Refrigerator 182lt 12-24V DW210
CategoryID: 20713
CategoryName: Refrigerators
Location: Italy
Country: IT
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 203
ItemID: 272531811428
Title: NEW Bosch KIN34P60AU 287L Serie 8 Integrated Bottom Mount Fridge
CategoryID: 20713
CategoryName: Refrigerators
Location: Sydney,NSW,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 204
ItemID: 303052658260
Title: DC285L
CategoryID: 

Current Item Index  : 301
ItemID: 233141783453
Title: Westinghouse 524L French Door Fridge - WHE5200SA-D
CategoryID: 20713
CategoryName: Refrigerators
Location: West Footscray,VIC,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 302
ItemID: 282139388541
Title: NEW Sharp SJXE676FSL 676L French Door Fridge
CategoryID: 20713
CategoryName: Refrigerators
Location: Sydney,NSW,Australia
Country: AU
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 303
ItemID: 362574233645
Title: Bosch Kgn36hi32 - Cool Freezer-Fridge Combination,Nofrost - Doors Stainless
CategoryID: 20713
CategoryName: Refrigerators
Location: Germany
Country: DE
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 304
ItemID: 282461741243
Title: Vitrifrigo Icemakers IM HYDRO XTP 12V Sta

Current Item Index  : 401
ItemID: 282461741222
Title: Vitrifrigo Icemakers IM CLASSIC REFILL P 115-230V Stainless steel #16006222, Mod
CategoryID: 20713
CategoryName: Refrigerators
Location: Italy
Country: IT
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 402
ItemID: 282461741290
Title: Vitrifrigo Stainless steel Drawer Refrigerator 75lt 12-24 Vdc 40W DW70 RFX #1600
CategoryID: 20713
CategoryName: Refrigerators
Location: Italy
Country: IT
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 403
ItemID: 133023738087
Title: AEG Sce81821lc - Cool Combi Fridge Freezer -
CategoryID: 20713
CategoryName: Refrigerators
Location: Germany
Country: DE
Condition: {'conditionId': '1000', 'conditionDisplayName': 'Brand New'}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Current Item Index  : 404
ItemID: 132990601747
Title: Siemens Ki85nad30 - Inbuilt Fridge Freezer C

...

{'itemId': '173474454278',
 'title': 'Mitsubishi 710L L4 Grande French Door Fridge MRL-710EG-DB=A',
 'categoryName': 'Refrigerators',
 'location': 'East Maitland,NSW,Australia',
 'country': 'AU',
 'viewItemURL': 'http://www.ebay.com.au/itm/Mitsubishi-710L-L4-Grande-French-Door-Fridge-MRL-710EG-DB-A-/173474454278',
 'condition': 'New: Never Used',
 'price': '2450.0',
 'currency': 'AUD',
 'brand': 'Mitsubishi',
 'capacity': 710,
 'model': 'MR-L710EG-PWH-A',
 'deliveryTime': None,
 'dispatchTimeMax': None}

{'itemId': '132977812414',
 'title': 'Siemens Kg56fsb40 - Cooling Freezer Combination, Nofrost - Doors Black',
 'categoryName': 'Refrigerators',
 'location': 'Germany',
 'country': 'DE',
 'viewItemURL': 'http://www.ebay.com.au/itm/Siemens-Kg56fsb40-Cooling-Freezer-Combination-Nofrost-Doors-Black-/132977812414',
 'condition': 'Brand New',
 'price': '2444.88',
 'currency': 'AUD',
 'brand': 'Siemens',
 'capacity': None,
 'model': 'Does Not Apply',
 'deliveryTime': '10',
 'dispatchTimeMax': '10'}

{'itemId': '263630082686',
 'title': 'REFRIGERATOR TWO DOORS 60CM HINGE RIGHT CREAM CLASS A FAB30RP1 SMEG 50 YEARS',
 'categoryName': 'Refrigerators',
 'location': 'Italy',
 'country': 'IT',
 'viewItemURL': 'http://www.ebay.com.au/itm/REFRIGERATOR-TWO-DOORS-60CM-HINGE-RIGHT-CREAM-CLASS-FAB30RP1-SMEG-50-YEARS-/263630082686',
 'condition': 'Brand New',
 'price': '2443.66',
 'currency': 'AUD',
 'brand': 'SMEG',
 'capacity': None,
 'model': None,
 'deliveryTime': '10',
 'dispatchTimeMax': '2'}

{'itemId': '273547428003',
 'title': 'NEW Sharp SJFJ676VBK 676L French Door Fridge',
 'categoryName': 'Refrigerators',
 'location': 'Sydney,NSW,Australia',
 'country': 'AU',
 'viewItemURL': 'http://www.ebay.com.au/itm/NEW-Sharp-SJFJ676VBK-676L-French-Door-Fridge-/273547428003',
 'condition': 'Brand New',
 'price': '2441.0',
 'currency': 'AUD',
 'brand': 'Sharp',
 'capacity': 676,
 'model': 'SJFJ676VBK',
 'deliveryTime': '3',
 'dispatchTimeMax': '3'}

{'itemId': '133031719501',
 'title': 'AEG Sce81864tc - Cool',
 'categoryName': 'Refrigerators',
 'location': 'Germany',
 'country': 'DE',
 'viewItemURL': 'http://www.ebay.com.au/itm/AEG-Sce81864tc-Cool-/133031719501',
 'condition': 'Brand New',
 'price': '2417.98',
 'currency': 'AUD',
 'brand': 'AEG',
 'capacity': None,
 'model': 'Does Not Apply',
 'deliveryTime': '10',
 'dispatchTimeMax': '10'}

In [ ]:
Daily_Call